In [ ]:
import sys,tweepy,csv,re
from textblob import TextBlob
import pandas as pd
import numpy as np

pd.set_option('display.notebook_repr_html', True)

class DataPreprocessing:

    def __init__(self):
        self.tweets = []
        self.tweetText = []
        self.polarity_class = []
        self.polarity_num = []
        temp = []
        
    def DataRetrieve(self):
        consumerKey = 'w56Mw69gHKd20VPb3LihtLPDS'
        consumerSecret = 'XpsKBB1Z8wpjCiRC9B3lpZoqmejVKpdmNeAmI1gPZsQlaHO7nc'
        
        accessToken = '186890076-xHXHdnBU81oLAXbUG44sUDcUxfdkq2eEG1wraWcu'
        accessTokenSecret = 'J696ThyY7lT3yVGfuDf43LUIgMqCffB1eY4ZXKWDqTDvK'
        
        auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
        auth.set_access_token(accessToken, accessTokenSecret)
        api = tweepy.API(auth)
        
        searchTerm = raw_input("\nEnter Keyword/Tag to search about: ")
        searchTermF = searchTerm + ' -filter:retweets'
        
        NoOfTerms = int(raw_input("Enter how many tweets to search: "))
        print ('Extracting...\n\n')
        
        self.tweets = tweepy.Cursor(api.search, q=searchTermF, lang='en').items(NoOfTerms)
        
        raw_tweet = []
        for tweet in self.tweets:
            raw_tweet.append(tweet.text)
            self.tweetText.append(self.cleanTweet(tweet.text).encode('utf-8'))
            analysis = TextBlob(tweet.text)
            self.polarity_num.append(format(analysis.sentiment.polarity, '.2f'))
            
        df_raw = pd.DataFrame()    
        df = pd.DataFrame()
        df_clean = pd.DataFrame()

        df_raw['Tweet'] = raw_tweet
        df_raw['Polarity_Value'] = self.polarity_num

        display (df_raw)

        print ('\n\nData after removing non-aplhabetical, hashtags, user mentions, link and etc : ')
        df['Tweet'] = self.tweetText
        df['Polarity_Value'] = self.polarity_num
        display (df)

        neu_count = 0
        pos_count = 0
        neg_count = 0

        for a in df.Polarity_Value:
            if float(a) == 0.00:
                self.polarity_class.append('Neutral')
                neu_count = neu_count + 1

            elif float(a) >= 0.01:
                self.polarity_class.append('Positive')
                pos_count = pos_count + 1

            elif float(a) < 0.00:
                self.polarity_class.append('Negative')
                neg_count = neg_count + 1


        df['Polarity_Class'] = self.polarity_class 

        for x in df.Polarity_Class: 
            if x == 'Neutral': 
                df_clean = df[df.Polarity_Class != 'Neutral']     

        df_clean = df_clean.sample(frac=1).reset_index(drop=True)

        print ('\n\n' +str(neu_count)+ ' tweet from neutral class label has been removed!')

        display(df_clean)
        #######################################################################  LET SAY YOU NAME THIS CSV FILE AS 'zidane'
        csvname = raw_input('\n\nPlease insert name of csv file to save : ')
        csvname = csvname+'.csv'
        df_clean.to_csv(csvname, sep=',', index=False, encoding='utf-8')
        print ('\nFile named ' +str(csvname)+ ' has been saved!')
        #######################################################################
        
    def cleanTweet(self, tweet):
            # Remove Links, Special Characters etc from tweet
            #tweet = str(tweet.correct())
            tweet = ' '.join(re.sub(r"http\S+", "",tweet).split())
            tweet = ' '.join(re.sub("@[A-Za-z0-9]+", "",tweet).split())
            tweet = ' '.join(re.sub("[^0-9A-Za-z \t]","",tweet).split())
            tweet = ' '.join(re.sub("\w +:\ / \ / \S +","",tweet).split())
            return tweet
        

if __name__== "__main__":
    sa = DataPreprocessing()
    sa.DataRetrieve() 


Enter Keyword/Tag to search about: bohemian rhapsody
Enter how many tweets to search: 500
Extracting...


